# Scrapeo de información
En este jupyter realizaremos el scrapeo en la página web de Ibis, de la información necesaria. Extraeremos la información sobre los nombres de los hoteles de la competencia, sus estrellas, y sus precios por noche. 

La web de la que sacaremos la información es la siguiente: https://ibis.accor.com/es/destination/city/hotels-madrid-v2418.html 

url con filtros aplicados (fechas estancias y disponibilidad): https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain 

In [1]:
# Importamos las librerías necesarias
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup as bs 
import requests
from selenium import webdriver # esto nos permite controlar navegadores web mediante código
from webdriver_manager.chrome import ChromeDriverManager # administra la instalación de chromedriver, que es necesario para controlar google chrome
from selenium.webdriver.chrome.service import Service # permite gestionar el servicio del driver de chrome
import time

In [4]:
# Inicializamos el WebDriver para Chrome

chromeService = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chromeService)

In [2]:
# Creamos la variable que contiene la url de la que queremos scrapear la información
url_ibis = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"
url_ibis

'https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain'

In [4]:
# abrimos la url especificada
driver.get(url_ibis)

In [11]:
# buscamos un elemento en concreto, en este caso buscamos el titulo de la clase title, el cual contiene el nombre del hotel así como las estrellas del mismo 
nombre = driver.find_elements("css selector", "div.hotelblock__content")
estrellas = driver.find_elements("css selector", "div.hotelblock__content-ratings")
precios = driver.find_elements("css selector", "div.rate-details__price-wrapper")

#__layout > div > main > div > div.hotels > div.search-list.search-list--max-width > div > div.search-list__content__main > section > div > div.hotelsList > section > div > div:nth-child(2) > ol > li:nth-child(1) > div > div > div.hotelblock__content > div.hotelblock__content-body > div.hotelblock__content-ratings > div.ratings > p

In [8]:
dict_hoteles = {"nombre": [],
                "estrellas": [],
                "precio": []}

In [7]:
for hotel in nombre:
    dict_hoteles["nombre"].append(hotel.text.split("\n")[0])

In [18]:
for estrella in estrellas:
    dict_hoteles["estrellas"].append(estrella.text.split("\n")[1].strip("/"))

In [9]:
for precio in precios:
    dict_hoteles["precio"].append(precio.text.split("\n")[1])

In [19]:
dict_hoteles

{'nombre': [],
 'estrellas': ['En',
  'En',
  'En',
  'En',
  'En',
  'En',
  'En',
  'En',
  'En',
  'En',
  'Puntuación de viajeros de TripAdvisor:\n4.7/\nEn\n5\nver reseñas\nIdeal para familias',
  'Puntuación de viajeros de TripAdvisor:\n4.4/\nEn\n5\nver reseñas',
  'Puntuación de viajeros de TripAdvisor:\n4.5/\nEn\n5\nver reseñas\nIdeal para familias',
  'Puntuación de viajeros de TripAdvisor:\n4.3/\nEn\n5\nver reseñas\nIdeal para familias',
  'Puntuación de viajeros de TripAdvisor:\n4.3/\nEn\n5\nver reseñas',
  'Puntuación de viajeros de TripAdvisor:\n4.4/\nEn\n5\nver reseñas\nIdeal para familias',
  'Puntuación de viajeros de TripAdvisor:\n4.4/\nEn\n5\nver reseñas\nIdeal para familias',
  'Puntuación de viajeros de TripAdvisor:\n4/\nEn\n5\nver reseñas',
  'Puntuación de viajeros de TripAdvisor:\n4.4/\nEn\n5\nver reseñas\nIdeal para familias',
  'Puntuación de viajeros de TripAdvisor:\n4.5/\nEn\n5\nver reseñas\nIdeal para familias',
  '4.7/',
  '4.4/',
  '4.5/',
  '4.3/',
  '4.3/

In [11]:
df_hoteles_comp = pd.DataFrame(dict_hoteles)
df_hoteles_comp

,nombre,estrellas,precio
0,ibis Styles Madrid Prado,3,170€
1,ibis Madrid Centro las Ventas,2,172€
2,ibis budget Madrid Vallecas,1,95€
3,ibis Madrid Aeropuerto Barajas,2,116€
4,ibis Madrid Alcorcon Tresaguas,2,90€
5,ibis budget Madrid Aeropuerto,1,97€
6,ibis Madrid Alcobendas,2,84€
7,ibis budget Madrid Alcorcon Móstoles,2,77€
8,ibis Madrid Fuenlabrada,2,86€
9,ibis Madrid Alcala de Henares La Garena,3,83€


In [9]:
def scrapeo_competencia(url):
    
    chromeService = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chromeService)

    driver.get(url_ibis)

    time.sleep(10)

    nombre_hotel = driver.find_elements("css selector", "div.hotelblock__content")
    estrellas_hotel = driver.find_elements("css selector", "div.hotelblock__content-ratings")
    precios_hotel = driver.find_elements("css selector", "div.rate-details__price-wrapper")

    diccionario_hoteles = {"nombre_hotel": [],
                "valoracion": [],
                "precio": []}
    
    for hotel in nombre_hotel:
        diccionario_hoteles["nombre_hotel"].append(hotel.text.split("\n")[0])
    
    for estrella in estrellas_hotel:
        diccionario_hoteles["valoracion"].append(estrella.text.split("\n")[1].strip("/"))
    
    for precio in precios_hotel:
        diccionario_hoteles["precio"].append(precio.text.split("\n")[1])

    hoteles_competencia = pd.DataFrame(diccionario_hoteles)
    hoteles_competencia["precio"] = hoteles_competencia["precio"].str.replace("€", "").astype(float)
    hoteles_competencia["valoracion"] = hoteles_competencia["valoracion"].astype(float)
    hoteles_competencia["fecha_reserva"] = pd.to_datetime("2025-02-21")
    
    hoteles_competencia.to_csv("../data/hoteles_competencia.csv", index = False)
    driver.close()

In [10]:
scrapeo_competencia(url_ibis)

In [12]:
df = pd.read_csv("../data/hoteles_competencia.csv", parse_dates = ["fecha_reserva"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_hotel   10 non-null     object        
 1   valoracion     10 non-null     float64       
 2   precio         10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 452.0+ bytes
